In [1]:
from numpy import fliplr, flipud
from skimage import img_as_ubyte
import itertools
import skimage.io
# from losses_pytorch import dice_loss
# import hausdorff
import warnings
from albumentations import Compose, HorizontalFlip
import albumentations as A
from torch.nn import MSELoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from IPython.display import YouTubeVideo
from IPython.display import clear_output
from IPython.display import Image as show_gif
import skimage
from skimage import util
from skimage.util import montage
from skimage.transform import resize
import imageio
# import seaborn as sns
# import matplotlib.gridspec as gridspec
# import matplotlib.patches as mpatches
# import matplotlib.animation as anim
# from matplotlib import cm
import matplotlib.pyplot as plt
# import h5py
# import nilearn.plotting as nlplt
# import nilearn as nl
import pydicom as pdm
import nibabel as nib
# from sklearn.model_selection import KFold
# from sklearn.svm import SVR
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import train_test_split
import pandas as pd
# from scipy import stats
import numpy as np
from random import randint
import time
import os
# from tqdm import tqdm
import torch
# import torchvision.transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from GPUtil import showUtilization as gpu_usage
from numba import cuda
from my_metric import dice_coef_metric,jaccard_coef_metric
from util.diceloss import DiceLoss


KernelSize = 3
KernelSizePad = 1
SamePadding = 2
Stride = 2
lr = 0.00001


class Double_conv(nn.Module):
    #conv->bn->relu->conv->bn->relu
    def __init__(self, in_ch, out_ch):
        super(Double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_ch, out_ch, kernel_size=KernelSize, padding=1),
            nn.BatchNorm3d(out_ch),
            # nn.GroupNorm(16,out_ch),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_ch, out_ch, kernel_size=KernelSize, padding=1),
            nn.BatchNorm3d(out_ch),
            # nn.GroupNorm(16,out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.maxpool = nn.Sequential(
            nn.MaxPool3d(kernel_size=2, stride=Stride),
            Double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        return self.maxpool(x)


class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Up, self).__init__()
        self.up = nn.ConvTranspose3d(in_ch,in_ch,kernel_size=2,stride=2)
        self.upcv = nn.Conv3d(in_ch, out_ch, kernel_size=3, padding=1)
        self.conv = Double_conv(in_ch, out_ch)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        x1 = self.upcv(x1)
        # diffZ = torch.tensor([x2.size()[2] - x1.size()[2]])
        # diffY = torch.tensor([x2.size()[3] - x1.size()[3]])
        # diffX = torch.tensor([x2.size()[4] - x1.size()[4]])
        # x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
        #                 diffY // 2, diffY - diffY // 2,
        #                 diffZ // 2, diffZ - diffZ // 2, ])

        x = torch.cat([x2, x1], dim=1)  # (B,C,D,H,W)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()
        self.conv = nn.Conv3d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class compVnet(nn.Module):
    def __init__(self, bilinear=True):
        super(compVnet, self).__init__()
        self.bilinear = bilinear
        self.inc = Double_conv(1, 16)
        self.down1 = Down(16, 32)
        self.down2 = Down(32, 64)
        self.down3 = Down(64, 128)
        self.down4 = Down(128, 256)
        self.up1 = Up(256, 128)
        self.up2 = Up(128, 64)
        self.up3 = Up(64, 32)
        self.up4 = Up(32, 16)
        self.outc = OutConv(16, 1)
        self.up1x = Up(256, 128)
        self.up2x = Up(128, 64)
        self.up3x = Up(64, 32)
        self.up4x = Up(32, 16)
        self.outcx = OutConv(16, 1)
        self.incz = Double_conv(1, 16)
        self.down1z = Down(16, 32)
        self.down2z = Down(32, 64)
        self.down3z = Down(64,128)
        # self.down4z = Down(128, 256)
        self.up1z = Up(256, 128)
        self.up2z = Up(128, 64)
        self.up3z = Up(64, 32)
        self.up4z = Up(32, 16)
        self.outcz = OutConv(16, 1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        # x5 = self.down4(x4)
        x = self.up2(x4, x3)
        # x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        y = self.up2x(x4, x3)
        # y = self.up2x(y, x3)
        y = self.up3x(y, x2)
        y = self.up4x(y, x1)
        y = self.outcx(y)
        # z = torch.cat([x, y], dim=1)
        z = torch.sigmoid(x)+torch.sigmoid(y)
        z1 = self.incz(z)
        z2 = self.down1z(z1)
        z3 = self.down2z(z2)
        z4 = self.down3z(z3)
        # z5 = self.down4z(z4)
        z = self.up2z(z4, z3)
        # z = self.up2z(z, z3)
        z = self.up3z(z, z2)
        z = self.up4z(z, z1)
        z = self.outcz(z)

        return x, y, z


/home/zhanghao/.conda/envs/ZH/lib/python3.9/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:

from util.oasisdataset3d import *
train_path, valid_path, test_path = getPathList()
train_loader = getDataloader(
    train_path, B1=1)
_,valid_loader,test_loader = getDataloader(
    train_path,valid_path,test_path
)
len(train_loader),len(valid_loader),len(test_loader)
#(1,1,160,224,224)

(340, 42, 43)

In [3]:
#全局权重初始化
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('conv3d') != -1:
        nn.init.kaiming_normal(m.weight)
        m.bias.data.zero()

#设置随机种子


def seed_everthing(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

In [4]:
from tensorboardX import SummaryWriter
def training(epochs, model, trainLoader, validLoader):
    dice_list = []
    seed_everthing(0)  # 设置全局随机种子
    lr = 1e-4
    dice_score = 0.
    iou_score = 0.
    writer = SummaryWriter('/data/zhanghao/skull_project/mae-main/base_line/compvnet_oasis_alldata')
    step = 0
    weights_init(model)  # 模型权重初始化
    Loss = DiceLoss()  # 实例化DiceLoss
    model.train()  # 设置model为训练模式

#____________________________________训练______________________________________________
    for epoch in range(epochs):
        # batch_num = 0  # 批次数
        optimizer = optim.Adam(model.parameters(), lr=lr)  # 每个epoch设置一次优化器
        model.train()  # 设置为训练模式
        optimizer.zero_grad()#优化器清零
        for data in trainLoader:  

            image = data["image"]  
            target = data["mask"]

            image = image.cuda(2)
            target = target.cuda(2)
            out = model(image)
            loss = Loss(out, target)
            loss.backward()
            optimizer.step()
            # print("step=",step)

            writer.add_scalar(tag="loss", scalar_value=loss, global_step=step)
            step += 1
        print("loss=", loss)

        optimizer.zero_grad()#在每轮训练后清零，给验证留出空间

#___________________________________验证_______________________________________________
        #每4个epoch进行一次验证
        if (epoch%4) == 0:
            with torch.no_grad():  # 表示在验证的时候不需要进行梯度计算
                model.eval()  # 设置model为验证模式
                dice_score = 0.
                iou_score = 0.
                for data in validLoader:  # validLoader共60个nii文件,batch_size=1
                    valid_image = data["image"]  
                    valid_target = data["mask"]

                    valid_image = valid_image.cuda(2)
                    valid_target = valid_target.cuda(2)
                    out = model(valid_image)
                    dice_score += dice_coef_metric(out.cpu(), valid_target.cpu())
                    iou_score += jaccard_coef_metric(out.cpu(), valid_target.cpu())
                    
                dice_score /= len(valid_loader)  # 算出平均dice_score
                iou_score /= len(valid_loader)


                print("epoch=", epoch,
                    "dice_score=", dice_score,
                    "iou_score=", iou_score)

                print("------------------------------------------------------------------")
                writer.add_scalar(tag="dice_scalar", scalar_value=dice_score, global_step=epoch)
                writer.add_scalar(tag="iou_scalar", scalar_value=iou_score, global_step=epoch)
                torch.save(model.state_dict(), '/data/zhanghao/skull_project/mae-main/base_line/compvnet_oasis_alldata/' +
                                    str(epoch) + 'Vnet' + str(dice_score) + '.pth')

        
            #早停
            if epoch > 20:
                min = 999
                for i in range(int(epoch/4 - 5) , int(epoch/4 - 1) ):
                    if dice_list[i] < min:
                        min = dice_list[i]
                if dice_score < min:
                    lr /= 10
                    print(lr)
                    if lr < 1e-6:
                        break

            dice_list.append(dice_score)#依次放入epoch为0 4 8 12 16 20 24....



In [5]:
print("len(train_loader)=", len(train_loader), "len(valid_loader)=", len(valid_loader))
model = compVnet()
model = model.cuda(2)  
# torch.backends.cudnn.enabled = True
# torch.backends.cudnn.benchmark = True
# training(epochs=700, model=model, trainLoader=train_loader, validLoader=valid_loader)

len(train_loader)= 340 len(valid_loader)= 42


In [6]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [8]:
training(epochs=700, model=model, trainLoader=train_loader, validLoader=valid_loader)

RuntimeError: Given transposed=1, weight of size [256, 256, 2, 2, 2], expected input[1, 128, 20, 28, 28] to have 256 channels, but got 128 channels instead